In [1]:
import pandas as pd
import numpy as np
import gzip
from imdb import IMDb


from bs4 import BeautifulSoup
import bs4
import requests
import re
import random

im = IMDb()

In [2]:
#title_filepath='name.basics.tsv.gz'
file_path='C:/Users/malav/Desktop/dat500-project/title.tsv'

In [3]:

#Extracting titleIDs from title_filepath
t_ids=[]
def get_ids(file_path):
    data=pd.read_csv(file_path,delimiter='\t')
    for i in data['tconst']:
        t_ids.append(str(i[2:9]))
    return(t_ids)

In [8]:
#Title summary (web scraped)
def get_summary(url):
    s=[]
    movie_page = requests.get(url)
    soup = BeautifulSoup(movie_page.text, 'html.parser')
    if (soup.find("div", class_="summary_text").contents):
        for i,text in enumerate(soup.find("div", class_="summary_text").contents):
            if (type(text)) != bs4.element.NavigableString:
                s.append((soup.find("div", class_="summary_text").contents)[i].contents[0])
            else:
                s.append((soup.find("div", class_="summary_text").contents)[i])
    s="".join(s)
    return(s)

In [ ]:
get_summary('https://www.imdb.com/title/tt0000008')

In [5]:
#Box office collection (web scraped)
def box_office(url):
    movie_page = requests.get(url)
    soup = BeautifulSoup(movie_page.text, 'html.parser')
    for a in soup.find_all("div", class_="txt-block"):
        if len(a.contents) > 2:
            for i in a.contents[1]:
                if i == 'Cumulative Worldwide Gross:':
                    return(a.contents[2])
                else:
                    return('-')

In [6]:
# 1) get_ids()
t_ids=get_ids(file_path)

In [9]:
data=[]

# 3) get_summary() and box_office()
for ID in t_ids[:10]:
    name=(im.get_movie(ID)['title'])
    url='https://www.imdb.com/title/tt'+ID
    summ=(get_summary(url))
    bo=(box_office(url))
    print()
    data.append(['tt'+ID,summ,bo])

df=pd.DataFrame(data)
df.columns=['tconst','Movie Summary','Box Office Collection']
df.to_csv('C:/Users/malav/Desktop/dat500-project/Summary_BO.tsv', sep = '\t')